In [1]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

lst_soorten_meta = ['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']
df_soorten_meta = pd.DataFrame(lst_soorten_meta).rename(columns={0: 'Soort'}) 

In [9]:
lst_tables = ['Def_Project', 'Def_Put', 'Def_Vondst', 'Def_Spoor', 'Def_Stelling', 'Def_Doos', 'Def_Standplaats', 'Def_Plaatsing', 'Def_Vlak', 'Def_Vindplaats', 'Def_Artefact'] #['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT origine, artefactsoort from "Def_Artefact"', connection)

df['origine'].unique()

array([None, 'Limburg', 'Eifel', 'Rijnland', 'Langewehe', 'Zuid-Limburg?',
       'Ardennen', 'Eifel, Mayen', 'Siegburg', 'Langerwehe',
       'Langerwehe?', 'Zuid-Limburg, Langerwehe', 'Rijnland, Siegburg?',
       'Siegburg?', 'Bergen op Zoom', 'Raeren / Frechen', 'Spaans',
       'Langerwehe/Raeren', 'Delft?', 'Locaal', 'wellicht Pingsdorf',
       'Zuid-Limburg, Langerwehe wellicht Coppengrave', 'Rijnalnd',
       'Rijnland, Langerwehe?', 'Nordrhein-Westfalen',
       'Langerwehe, Raeren', 'Raeren', 'Rijnland, Aken?',
       'Rijnland, Siegburg', 'andenne', 'andenne?', 'Frechen',
       'Aken, Raeren', 'Baumberg ?', 'Frechen / Raeren', 'Sauerland',
       'Keulen', 'Bergen op Zoom?sluitpan/gortpan', 'Westerwald',
       'China, Jingdezhen', '?', 'lokaal', 'Delft', 'Maastricht',
       'Holland', 'West-Friesland', 'Overijssel', 'Zeeland', 'Friesland',
       'Rome', 'Utrecht', 'Vlaanderen', 'Delft ?', 'duitsland',
       'Frankrijk', 'Duitsland?', 'China,Jingdezhen?', 'Europa?',
   